In [20]:
import os
from dotenv import load_dotenv

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document



In [14]:
load_dotenv(dotenv_path="../.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USERNAME")
PWD = os.getenv("NEO4J_PASSWORD")
graph = Neo4jGraph(url=URI, username=USER, password=PWD)

In [15]:
text = """
Ada Lovelace was a mathematician who worked with Charles Babbage on the Analytical Engine.
She is often credited as the first computer programmer.
"""

In [18]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(llm=llm)



In [21]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='first computer programmer', type='Title', properties={}), Node(id='Ada Lovelace', type='Person', properties={}), Node(id='Charles Babbage', type='Person', properties={}), Node(id='Analytical Engine', type='Invention', properties={})]
Relationships:[Relationship(source=Node(id='Ada Lovelace', type='Person', properties={}), target=Node(id='Charles Babbage', type='Person', properties={}), type='WORKED_WITH', properties={}), Relationship(source=Node(id='Ada Lovelace', type='Person', properties={}), target=Node(id='Analytical Engine', type='Invention', properties={}), type='WORKED_ON', properties={}), Relationship(source=Node(id='Ada Lovelace', type='Person', properties={}), target=Node(id='first computer programmer', type='Title', properties={}), type='CREDITED_AS', properties={})]


In [22]:
graph.add_graph_documents(graph_documents)

In [23]:
graph.query("MATCH (n) RETURN count(n) AS TOTAL")  # Example query to count nodes

[{'TOTAL': 4}]

In [26]:
graph.query("MATCH (a:Person) RETURN a")

[{'a': {'id': 'Ada Lovelace'}}, {'a': {'id': 'Charles Babbage'}}]

In [27]:
graph.query('MATCH (n) DETACH DELETE n')

[]